## Importación de bibliotecas, funciones necesarias y carga de modelos



In [44]:
import pandas as pd
from collections import Counter, defaultdict
import numpy as np
import pickle
import math
import random

def crearDefaultdictCounter():
    """
    Crea un defaultdict de Counter, para utilizar en el modelo de trigramas
    
    Retorna:
    defaultdict(Counter): Un defaultdict que retorna un Counter por defecto
    """
    return defaultdict(Counter)

def cargarModelo(nombreArchivo):
    """
    Carga un modelo desde un archivo pickle.
    
    Parámetros:
    - nombreArchivo (str): Nombre del archivo desde donde se cargará el modelo
    
    Retorna:
    - modelo: El modelo cargado.
    """
    with open(nombreArchivo, 'rb') as archivo:
        modelo = pickle.load(archivo)
    return modelo

def separarOraciones(datos):
    """
    Separar las oraciones de un DataFrame a partir de la columna 'unique'
    
    Parámetros:
    - datos (pd.DataFrame): DataFrame que contiene una columna 'unique' con las oraciones
    
    Retorna
    - list: Lista de oraciones tokenizadas, donde cada oración es una lista de tokens
    """
    oracionesSeparadas = []
    for oracion in datos['unique']:
        tokens = oracion.split()  
        oracionesSeparadas.append(tokens)
    return oracionesSeparadas



In [45]:
#20N
unigrama20N = cargarModelo('20N_01_unigramas.pkl')
bigrama20N = cargarModelo('20N_01_bigramas.pkl')
trigrama20N = cargarModelo('20N_01_trigramas.pkl')

#BAC
unigramaBAC = cargarModelo('BAC_01_unigramas.pkl')
bigramaBAC = cargarModelo('BAC_01_bigramas.pkl')
trigramaBAC = cargarModelo('BAC_01_trigramas.pkl')

# testing csvs
datos20N_test = pd.read_csv('20N_01_testing.csv')
datosBAC_test = pd.read_csv('BAC_01_testing.csv')
oracionesSeparadas20N_test = separarOraciones(datos20N_test)
oracionesSeparadasBAC_test = separarOraciones(datosBAC_test)

tamanoVocabulario20N = len(unigrama20N)
tamanoVocabularioBAC = len(unigramaBAC)


## Cálculo de Perplejidad para los Modelos N-Grama

Para wvaluar el rendimiento de nuestros modelos de lenguaje, calculamos la **perplejidad** de cada uno sobre el conjunto de prueba. La perplejidad es una medida que indica qué tan bien un modelo predice una muestra. Un valor de perplejidad más bajo sugiere un modelo mejor

Definimos funciones para calcular la perplejidad de los modelos de **unigramas**, **bigramas** y **trigramas**


In [46]:
def calcularPerplejidadUnigrama(modeloUnigrama, oracionesSeparadas, tamanoVocabulario):
    """
    Calcula la perplejidad de un modelo de unigramas dado un conjunto de oraciones tokenizadas

    Parámetros:
    - modeloUnigrama (Counter): Modelo de unigramas
    - oracionesSeparadas (list): Lista de oraciones tokenizadas
    - tamanoVocabulario (int): Tamaño del vocabulario del modelo

    Retorna:
    - float: Perplejidad del modelo sobre el conjunto de oracione
    """
    totalLogProb = 0
    totalPalabras = 0
    totalFrecuencia = sum(modeloUnigrama.values())

    for oracion in oracionesSeparadas:
        for palabra in oracion:
            frecuencia = modeloUnigrama.get(palabra, 0)
            probabilidad = (frecuencia + 1) / (totalFrecuencia + tamanoVocabulario)
            logProb = math.log(probabilidad)
            totalLogProb += logProb
            totalPalabras += 1

    perplejidad = math.exp(-totalLogProb / totalPalabras)
    return perplejidad

def calcularPerplejidadBigrama(modeloBigrama, modeloUnigrama, oracionesSeparadas, tamanoVocabulario):
    """
    Calcula la perplejidad de un modelo de bigramas dado un conjunto de oraciones tokenizadas

    Parámetros:
    - modeloBigrama (defaultdict(Counter)): Modelo de bigramas
    - modeloUnigrama (Counter): Modelo de unigramas
    - oracionesSeparadas (list): Lista de oraciones tokenizqdas
    - tamanoVocabulario (int): Tamaño del vocabulario del modelo

    Retorna:
    - float: Perplejidad del modelo sobre el conjunto de oraciones
    """
    totalLogProb = 0
    totalPalabras = 0

    for oracion in oracionesSeparadas:
        for i in range(len(oracion) - 1):
            palabra1 = oracion[i]
            palabra2 = oracion[i+1]

            frecuenciaBigram = modeloBigrama.get(palabra1, {}).get(palabra2, 0)
            frecuenciaUnigram = modeloUnigrama.get(palabra1, 0)
            probabilidad = (frecuenciaBigram + 1) / (frecuenciaUnigram + tamanoVocabulario)
            logProb = math.log(probabilidad)
            totalLogProb += logProb
            totalPalabras += 1

    perplejidad = math.exp(-totalLogProb / totalPalabras)
    return perplejidad

def calcularPerplejidadTrigrama(modeloTrigrama, modeloBigrama, oracionesSeparadas, tamanoVocabulario):
    """
    Calcula la perplejidad de un modelo de trigramas dado un conjunto de oraciones tokenizadas

    Parámetros:
    - modeloTrigrama (defaultdict): Modelo de trigramas
    - modeloBigrama (defaultdict(Counter)): Modelo de bigramas
    - oracionesSeparadas (list): Lista de oraciones tokenizadas

    Retorna:
    - float: Perplejidad del modelo sobre el conjunto de oraciones
    """
    totalLogProb = 0
    totalPalabras = 0

    for oracion in oracionesSeparadas:
        for i in range(len(oracion) - 2):
            palabra1 = oracion[i]
            palabra2 = oracion[i+1]
            palabra3 = oracion[i+2]

            frecuenciaTrigram = modeloTrigrama.get(palabra1, {}).get(palabra2, {}).get(palabra3, 0)
            frecuenciaBigram = modeloBigrama.get(palabra1, {}).get(palabra2, 0)
            probabilidad = (frecuenciaTrigram + 1) / (frecuenciaBigram + tamanoVocabulario)
            logProb = math.log(probabilidad)
            totalLogProb += logProb
            totalPalabras += 1

    perplejidad = math.exp(-totalLogProb / totalPalabras)
    return perplejidad


## Resultados de la Perplejidad

Después de calcular las perplejidades, organizamos los resultados en una tabla 


In [47]:
# Calcular perplejidad para 20N
perplejidadUnigrama20N = calcularPerplejidadUnigrama(unigrama20N, oracionesSeparadas20N_test, tamanoVocabulario20N)
perplejidadBigrama20N = calcularPerplejidadBigrama(bigrama20N, unigrama20N, oracionesSeparadas20N_test, tamanoVocabulario20N)
perplejidadTrigrama20N = calcularPerplejidadTrigrama(trigrama20N, bigrama20N, oracionesSeparadas20N_test, tamanoVocabulario20N)

# Calcular perplejidad para BAC
perplejidadUnigramaBAC = calcularPerplejidadUnigrama(unigramaBAC, oracionesSeparadasBAC_test, tamanoVocabularioBAC)
perplejidadBigramaBAC = calcularPerplejidadBigrama(bigramaBAC, unigramaBAC, oracionesSeparadasBAC_test, tamanoVocabularioBAC)
perplejidadTrigramaBAC = calcularPerplejidadTrigrama(trigramaBAC, bigramaBAC, oracionesSeparadasBAC_test, tamanoVocabularioBAC)

# Mostrar resultados
print("Perplejidades calculadas:\n")
print(f"Modelo Unigrama 20N: {perplejidadUnigrama20N}")
print(f"Modelo Bigrama 20N: {perplejidadBigrama20N}")
print(f"Modelo Trigrama 20N: {perplejidadTrigrama20N}\n")

print(f"Modelo Unigrama BAC: {perplejidadUnigramaBAC}")
print(f"Modelo Bigrama BAC: {perplejidadBigramaBAC}")
print(f"Modelo Trigrama BAC: {perplejidadTrigramaBAC}")

Perplejidades calculadas:

Modelo Unigrama 20N: 1514.136822471852
Modelo Bigrama 20N: 3119.2630248796004
Modelo Trigrama 20N: 18635.35423630719

Modelo Unigrama BAC: 1123.649051033008
Modelo Bigrama BAC: 1451.2946974119318
Modelo Trigrama BAC: 21615.39448207005


In [48]:
resultados = pd.DataFrame({
    'Modelo': ['Unigrama', 'Bigrama', 'Trigrama'],
    'Perplejidad 20N': [perplejidadUnigrama20N, perplejidadBigrama20N, perplejidadTrigrama20N],
    'Perplejidad BAC': [perplejidadUnigramaBAC, perplejidadBigramaBAC, perplejidadTrigramaBAC]
})

print("Tabla de perplejidades:\n")
print(resultados)


Tabla de perplejidades:

     Modelo  Perplejidad 20N  Perplejidad BAC
0  Unigrama      1514.136822      1123.649051
1   Bigrama      3119.263025      1451.294697
2  Trigrama     18635.354236     21615.394482


- El **modelo de unigramas** tiene la perplejidad más baja en ambos datasets, lo que indica que, desde una perspectiva de predicción, es el más efectivo según esta métrica
- Los **modelos de bigramas y trigramas** presentan perplejidades más altas, posiblemente debido a la escasez de datos para ciertas combinaciones de palabras

Sin embargo, es importante considerar que aunque el modelo de unigramas tiene la perplejidad más baja no captura las dependencias entre palabras, lo cual es esencial para generar oraciones coherentes!!!


## Generación de Oraciones Utilizando el Mejor Modelo (Modelo Unigrama)

De acuerdo con los resultados obtenidos, el **modelo de unigramas** presenta la perplejidad más baja en ambos datasets, por lo que es considerado el mejor modelo según esta métrica.

Aunque los modelos de unigramas no capturan las dependencias entre palabras, utilizaremos este modelo para generar oraciones porque lo dicen en elenunciado

Definimos una función que, dado un modelo de unigramas y una palabra inicial, genera una oración de manera automática.


In [49]:
def generarSentenciaUnigrama(modeloUnigrama, palabraInicial, tamanoVocabulario, maxLongitud=20):
    """
    Genera una oración utilizando el modelo de unigramas, comenzando con una palabra dada
    
    Parámetros:
    - modeloUnigrama (Counter): Modelo de unigramas
    - palabraInicial (str): Palabra inicial de la oración
    - tamanoVocabulario (int): Tamaño del vocabulario
    - maxLongitud (int): Longitud máxima de la oración generad
    
    Retorna:
    - str: Oración generada.
    """
    oracion = [palabraInicial]
    
    totalFrecuencia = sum(modeloUnigrama.values()) + tamanoVocabulario
    palabras = list(modeloUnigrama.keys())
    frecuencias = [modeloUnigrama.get(palabra, 0) + 1 for palabra in palabras]  
    
    probabilidades = np.array(frecuencias) / totalFrecuencia
    
    for _ in range(maxLongitud - 1):
        palabraSiguiente = np.random.choice(palabras, p=probabilidades)
        oracion.append(palabraSiguiente)
        
        # Si la palabra es el token de fin de oración, detenemos la generación
        if palabraSiguiente == '</s>':
            break
    
    return ' '.join(oracion)


## Pruebas de Generación de Oraciones y Análisis de Resultados

Probamos la función de generación de oraciones con diferentes palabras iniciales para evaluar cómo el modelo de unigramas genera texto. Algunas de las palabras iniciales utilizadas son:

- `<s>` (inicio de oración)
- `the`
- `NUM`
- `this`
- `it`
- `i`
- `he`
- `she`
- `they`
- `we`

Mostramos las oraciones generadas y analizamos su coherencia y características.


In [50]:
# Lista de palabras iniciales para probar
palabrasIniciales = ['<s>', 'the', 'NUM', 'this', 'it', 'i', 'he', 'she', 'they', 'we']

print("Generación de oraciones utilizando el modelo Unigrama:\n")

for palabraInicial in palabrasIniciales:
    if palabraInicial not in unigrama20N:
        palabraInicial = '<UNK>'
    
    oracionGenerada = generarSentenciaUnigrama(unigrama20N, palabraInicial, tamanoVocabulario20N)
    
    print(f"Palabra inicial: '{palabraInicial}'")
    print(f"Oración generada: {oracionGenerada}\n")



Generación de oraciones utilizando el modelo Unigrama:

Palabra inicial: '<s>'
Oración generada: <s> on require the an public on the can foil nose you corrections wouldn it you refugees discusses NUM i

Palabra inicial: 'the'
Oración generada: the be of me home were <UNK> point division faq oscillators performance could bursts NUM ziemans get NUM cases let

Palabra inicial: 'NUM'
Oración generada: NUM missions allegedly sale my kuwait priorities scott ax tj there access edu february the ___ silent before co of

Palabra inicial: 'this'
Oración generada: this you check america jews <s> have lives all intent acceptable kept past intents seat introduction mi0 date adobe matter

Palabra inicial: 'it'
Oración generada: it edu we car that fortune m immune constrains pacific NUM the the for very about in local <UNK> in

Palabra inicial: 'i'
Oración generada: i e i so way good is tongues for what a rodrigues disabled at citizen simple however soest r for

Palabra inicial: 'he'
Oración generada:

### Análisis de las Oraciones Generadas

Al revisar las oraciones generadas por el **modelo de unigramas**, observamos que:

- Las palabras aparecen de manera independiente, sin considerar el contexto previo
- Las oraciones suelen no tener de coherencia gramatical y semántica
- Es común que las oraciones sean simplemente secuencias aleatorias de palabras del vocabulario, reflejando las frecuencias individuales de las palabras en el corpus.

 

## Generación de Oraciones Utilizando el Modelo Trigrama

Con el objetivo de mejorar la coherencia y contexto de las oraciones generadas, implementamos una función para generar oraciones utilizando el **modelo de trigramas**. Al considerar dos palabras previas, el modelo trigrama puede capturar dependencias más largas.

Realizamos pruebas similares a las anteriores y comparamos los resultados.


In [51]:
def generarSentenciaTrigrama(modeloTrigrama, modeloBigrama, palabraInicial, tamanoVocabulario, maxLongitud=20):
    """
    Genera una oración utilizando el modelo de trigramas, comenzando con una palabra dada

    Parámetros:
    - modeloTrigrama (defaultdict): Modelo de trigramqs
    - modeloBigrama (defaultdict(Counter)): Modelo de bigramas
    - palabraInicial (str): Palabra inicial de la oración
    - tamanoVocabulario (int): Tamaño del vocabulario
    - maxLongitud (int): Longitud máxima de la oración generad

    Retorna:
    - str: Oración generada.
    """
    oracion = [palabraInicial]

    posiblesSiguientes = modeloBigrama.get(palabraInicial, None)
    if not posiblesSiguientes:
        return ' '.join(oracion)

    totalFrecuencia = sum(posiblesSiguientes.values()) + tamanoVocabulario
    palabras = []
    probabilidades = []

    for palabra, frecuencia in posiblesSiguientes.items():
        palabras.append(palabra)
        probabilidad = (frecuencia + 1) / totalFrecuencia
        probabilidades.append(probabilidad)

    probabilidades = np.array(probabilidades)
    probabilidades /= probabilidades.sum()

    palabraSiguiente = np.random.choice(palabras, p=probabilidades)
    oracion.append(palabraSiguiente)

    for _ in range(maxLongitud - 2):
        palabra1 = oracion[-2]
        palabra2 = oracion[-1]
        posiblesSiguientes = modeloTrigrama.get(palabra1, {}).get(palabra2, None)

        if not posiblesSiguientes:
            break

        totalFrecuencia = sum(posiblesSiguientes.values()) + tamanoVocabulario
        palabras = []
        probabilidades = []

        for palabra, frecuencia in posiblesSiguientes.items():
            palabras.append(palabra)
            probabilidad = (frecuencia + 1) / totalFrecuencia
            probabilidades.append(probabilidad)

        probabilidades = np.array(probabilidades)
        probabilidades /= probabilidades.sum()

        palabraSiguiente = np.random.choice(palabras, p=probabilidades)
        oracion.append(palabraSiguiente)

        # Si la palabra es el token de fin de oración, detenemos la generación
        if palabraSiguiente == '</s>':
            break

    return ' '.join(oracion)




In [52]:
print("Generación de oraciones utilizando el modelo Trigrama:\n")

for palabraInicial in palabrasIniciales:
    if palabraInicial not in unigrama20N:
        palabraInicial = '<UNK>'

    oracionGenerada = generarSentenciaTrigrama(trigrama20N, bigrama20N, palabraInicial, tamanoVocabulario20N)

    print(f"Palabra inicial: '{palabraInicial}'")
    print(f"Oración generada: {oracionGenerada}\n")


Generación de oraciones utilizando el modelo Trigrama:

Palabra inicial: '<s>'
Oración generada: <s> kime mongoose torolab ibm com ricardo rchland vnet ibm com internal mjp bwa kgn ibm com nareid helge helge

Palabra inicial: 'the'
Oración generada: the day following the end section x build xbegin NUM prog c x i am not sufficiently proved such borderline

Palabra inicial: 'NUM'
Oración generada: NUM NUM NUM minnesota vs vancouver saskatoon tue NUM apr NUM NUM z4jn wit NUM jesus himself be he should

Palabra inicial: 'this'
Oración generada: this utter nonsense if so will this kind of self defense purposes it s quiet on the same caliber with

Palabra inicial: 'it'
Oración generada: it is also the team <UNK> us r6 t4 mt44 y8 p 13s1 NUM qs tpi gg l NUM NUM

Palabra inicial: 'i'
Oración generada: i mistaken with this exact same phenomenon occurs with my lciii perhaps it is quite lightweight and easy configuration i

Palabra inicial: 'he'
Oración generada: he never reply again <UNK> are eff

### Análisis de las Oraciones Generadas con el Modelo Trigrama

Las oraciones generadas con el modelo trigrama muestran:

- Una ligerq mejora en la coherencia, gracias a la consideración de un contexto más amplio
- Persistencia de algunas limitaciones, como repeticiones y posibles incoherencias

